In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import statsmodels.formula.api as smf
from yahoo_fin import stock_info as si

In [8]:
# Pulls daily adjusted close prices for one ticker. Returns: date, price, ticker
def get_daily_prices(ticker, start_date, end_date):
    df = si.get_data(ticker, start_date=start_date, end_date=end_date).reset_index()
    df["price"] = df["adjclose"]
    df["ticker"] = ticker
    return df

def get_prices(tickerlist, start_date, end_date):
    df = pd.DataFrame(columns=["date", "price", "ticker"])
    for ticker in tickerlist:
        df = pd.concat(
            [df, get_daily_prices(ticker, start_date, end_date)],
            ignore_index=True
        )
    return df
